In [ ]:
#### Libraries ####
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import ipywidgets as widgets
import qgrid
import pandas_profiling

In [ ]:
#### Settings ####
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 400) 
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_colwidth', -1)
%matplotlib inline
sns.set_style("whitegrid")
sns.set_context("notebook")


In [ ]:
#### Function Library ####
def examine_df(df):
    print(df.info())
    print(df.shape)
    print(df.head())
    print(df.describe())
    
    return pandas_profiling.ProfileReport(df)

def write_result_to_csv(df):
    df.to_csv('result.csv', sep=',')
    
def distribution_plot(df, column_name):
    sns.distplot(df[column_name])
    return plt.figure()
    
def joint_plot(df,x,y):
    return sns.jointplot(x = x, y=y, data = df, kind = 'reg')

def pair_plot(df):
    return sns.pairplot(df, kind = 'reg')

    


In [ ]:
#### Data Import ####

In [ ]:
### Calculations ### 